# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 47 new papers today
          22 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/21 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2208.08997


extracting tarball to tmp_2208.08997...

 done.
Retrieving document from  https://arxiv.org/e-print/2208.09000


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: LatexWarning: Found documentclass in tmp_2208.08997/thepaper_final.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'table_mass_comparison' from 'tmp_2208.08997/table_mass_comparison.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'table_mass' from 'tmp_2208.08997/table_mass.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.13/x64/lib/

...

 done.


Retrieving document from  https://arxiv.org/e-print/2208.09369


extracting tarball to tmp_2208.09369... done.
Retrieving document from  https://arxiv.org/e-print/2208.09377


extracting tarball to tmp_2208.09377...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.09335-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.09335) | **An empirical model of the Gaia DR3 selection function**  |
|| <mark>Tristan Cantat-Gaudin</mark>, et al. -- incl., <mark>Morgan Fouesneau</mark>, <mark>Hans-Walter Rix</mark>, <mark>Anthony G. A. Brown</mark>, <mark>Gregory M. Green</mark> |
|*Appeared on*| *2022-08-22*|
|*Comments*| *to be submitted to A&A*|
|**Abstract**| Interpreting and modelling astronomical catalogues requires an understanding of the catalogues' completeness or selection function: objects of what properties had a chance to end up in the catalogue. Here we set out to empirically quantify the completeness of the overall Gaia DR3 catalogue. This task is not straightforward because Gaia is the all-sky optical survey with the highest angular resolution to date and no consistent ``ground truth'' exists to allow direct comparisons. However, well-characterised deeper imaging enables an empirical assessment of Gaia's $G$-band completeness across parts of the sky. On this basis, we devised a simple analytical completeness model of Gaia as a function of the observed $G$ magnitude and position over the sky, which accounts for both the effects of crowding and the complex Gaia scanning law. Our model only depends on a single quantity: the median magnitude $M_{10}$ in a patch of the sky of catalogued sources with $\texttt{astrometric_matched_transits}$ $\leq 10$. $M_{10}$ reflects elementary completeness decisions in the Gaia pipeline and is computable from the Gaia DR3 catalogue itself and therefore applicable across the whole sky. We calibrate our model using the Dark Energy Camera Plane Survey (DECaPS) and test its predictions against Hubble Space Telescope observations of globular clusters. We find that our model predicts Gaia's completeness values to a few per cent across the sky. We make the model available as a part of the $\texttt{gaiasf}$ Python package built and maintained by the GaiaUnlimited project. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.09008-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.09008) | **SDSS-IV MaNGA: the chemical co-evolution of gas and stars in spiral  galaxies**  |
|| <mark>Michael J. Greener</mark>, et al. -- incl., <mark>Joel R. Brownstein</mark> |
|*Appeared on*| *2022-08-22*|
|*Comments*| *15 pages, 8 figures, accepted for publication by MNRAS*|
|**Abstract**| We investigate archaeologically how the metallicity in both stellar and gaseous components of spiral galaxies of differing masses evolve with time, using data from the SDSS-IV MaNGA survey. For the stellar component, we can measure this evolution directly by decomposing the galaxy absorption-line spectra into populations of different ages and determining their metallicities. For the gaseous component, we can only measure the present-day metallicity directly from emission lines. However, there is a well-established relationship between gas metallicity, stellar mass and star formation rate which does not evolve significantly with redshift; since the latter two quantities can be determined directly for any epoch from the decomposition of the absorption-line spectra, we can use this relationship to infer the variation in gas metallicity over cosmic time. Comparison of present-day values derived in this way with those obtained directly from the emission lines confirms the validity of the method. Application of this approach to a sample of 1619 spiral galaxies reveals how the metallicity of these systems has changed over the last 10 billion years since cosmic noon. For lower-mass galaxies, both stellar and gaseous metallicity increase together, as one might expect in well-mixed fairly isolated systems. In higher-mass systems, the average stellar metallicity has not increased in step with the inferred gas metallicity, and actually decreases with time. Such disjoint behaviour is what one might expect if these more massive systems have accreted significant amounts of largely pristine gas over their lifetimes, and this material has not been well mixed into the galaxies. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.08997-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.08997) | **New Velocity Measurements of NGC 5128 Globular Clusters out to 130 kpc:  Outer Halo Kinematics, Substructure and Dynamics**  |
|| <mark>A.K. Hughes</mark>, et al. |
|*Appeared on*| *2022-08-22*|
|*Comments*| *24 pages, 13 figures, submitted to AAS Journals*|
|**Abstract**| We present new radial velocity measurements from the Magellan and the Anglo-Australian Telescopes for 174 previously known and 122 newly confirmed globular clusters (GCs) around NGC 5128, the nearest accessible massive early-type galaxy at D=3.8 Mpc. Remarkably, 28 of these newly confirmed GCs are at projected radii >50' ($\gtrsim 54$ kpc), extending to $\sim 130$ kpc, in the outer halo where few GCs had been confirmed in previous work. We identify several subsets of GCs that spatially trace halo substructures that are visible in red giant branch star maps of the galaxy. In some cases, these subsets of GCs are kinematically cold, and may be directly associated with and originate from these specific stellar substructures. From a combined kinematic sample of 645 GCs, we see evidence for coherent rotation at all radii, with a higher rotation amplitude for the metal-rich GC subpopulation. Using the tracer mass estimator, we measure a total enclosed mass of $2.5\pm0.3 \times 10^{12} M_{\odot}$ within $\sim 120$ kpc, an estimate that will be sharpened with forthcoming dynamical modeling. The combined power of stellar mapping and GC kinematics makes NGC 5128 an ongoing keystone for understanding galaxy assembly at mass scales inaccessible in the Local Group. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.09272-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.09272) | **$E_{\mathrm{iso}}$-$E_{\mathrm{p}}$ correlation of gamma ray bursts:  calibration and cosmological applications**  |
|| X. D. Jia, et al. -- incl., <mark>B. B. Zhang</mark> |
|*Appeared on*| *2022-08-22*|
|*Comments*| *14 pages, 7 figures, 3 tables, accepted for publication in MNRAS*|
|**Abstract**| Gamma-ray bursts (GRBs) are the most explosive phenomena and can be used to study the expansion of Universe. In this paper, we compile a long GRB sample for the $E_{\mathrm{iso}}$-$E_{\mathrm{p}}$ correlation from Swift and Fermi observations. The sample contains 221 long GRBs with redshifts from 0.03 to 8.20. From the analysis of data in different redshift intervals, we find no statistically significant evidence for the redshift evolution of this correlation. Then we calibrate the correlation in six sub-samples and use the calibrated one to constrain cosmological parameters. Employing a piece-wise approach, we study the redshift evolution of dark energy equation of state (EOS), and find that the EOS tends to be oscillating at low redshift, but consistent with $-1$ at high redshift. It hints a dynamical dark energy at $2\sigma$ confidence level at low redshift. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.09369-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.09369) | **Dynamical cluster masses from photometric surveys**  |
|| Omar Contigiani, et al. -- incl., <mark>Henk Hoekstra</mark> |
|*Appeared on*| *2022-08-22*|
|*Comments*| **|
|**Abstract**| The masses of galaxy clusters can be measured using data obtained exclusively from wide photometric surveys in one of two ways: directly from the amplitude of the weak lensing signal or, indirectly, through the use of scaling relations calibrated using binned lensing measurements. In this paper, we build on a recently proposed idea and implement an alternative method based on the radial profile of the satellite distribution. This technique relies on splashback, a feature associated with the apocenter of recently accreted galaxies that offers a clear window into the phase-space structure of clusters without the use of velocity information. We carry out this dynamical measurement using the stacked satellite distribution around a sample of luminous red galaxies in the fourth data release of the Kilo-Degree Survey and validate our results using abundance-matching and lensing masses. To illustrate the power of this measurement, we combine dynamical and lensing mass estimates to robustly constrain scalar-tensor theories of gravity at cluster scales. Our results exclude departures from General Relativity of order unity. We conclude the paper by discussing the implications for future data sets. Because splashback mass measurements scale only with the survey volume, stage-IV photometric surveys are well-positioned to use splashback to provide high-redshift cluster masses. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.09377-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.09377) | **J-PLUS: Discovery and characterisation of ultracool dwarfs using Virtual  Observatory tools II. Second data release and machine learning methodology**  |
|| P. Mas-Buitrago, et al. -- incl., <mark>C. Hernández-Monteagudo</mark>, <mark>H. Vázquez Ramió</mark> |
|*Appeared on*| *2022-08-22*|
|*Comments*| *Accepted in A&A*|
|**Abstract**| Ultracool dwarfs (UCDs) comprise the lowest mass members of the stellar population and brown dwarfs, from M7 V to cooler objects with L, T, and Y spectral types. Most of them have been discovered using wide-field imaging surveys, for which the Virtual Observatory (VO) has proven to be of great utility. We aim to perform a search for UCDs in the entire Javalambre Photometric Local Universe Survey (J-PLUS) second data release (2176 deg$^2$) following a VO methodology. We also explore the ability to reproduce this search with a purely machine learning (ML)-based methodology that relies solely on J-PLUS photometry. We followed three different approaches based on parallaxes, proper motions, and colours, respectively, using the VOSA tool to estimate the effective temperatures. For the ML methodology, we built a two-step method based on principal component analysis and support vector machine algorithms. We identified a total of 7827 new candidate UCDs, which represents an increase of about 135% in the number of UCDs reported in the sky coverage of the J-PLUS second data release. Among the candidate UCDs, we found 122 possible unresolved binary systems, 78 wide multiple systems, and 48 objects with a high Bayesian probability of belonging to a young association. We also identified four objects with strong excess in the filter corresponding to the Ca II H and K emission lines and four other objects with excess emission in the H$\alpha$ filter. With the ML approach, we obtained a recall score of 92% and 91% in the test and blind test, respectively. We consolidated the proposed search methodology for UCDs, which will be used in deeper and larger upcoming surveys such as J-PAS and Euclid. We concluded that the ML methodology is more efficient in the sense that it allows for a larger number of true negatives to be discarded prior to analysis with VOSA, although it is more photometrically restrictive. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.09013-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.09013) | **Forecasting constraints on the mean free path of ionizing photons at $z  \geq 5.4$ from the Lyman-$α$ forest flux auto-correlation function**  |
|| Molly Wolfson, <mark>Joseph F. Hennawi</mark>, <mark>Frederick B. Davies</mark>, Jose Oñorbe |
|*Appeared on*| *2022-08-22*|
|*Comments*| *Submitted to MNRAS, 16 pages, 10 figures*|
|**Abstract**| Lyman-$\alpha$ (Ly$\alpha$) forest transmission towards high-$z$ quasars contains information on the state of the universe just after the epoch of reionization. Fluctuations in Ly$\alpha$ forest transmission are partially sourced from spatial fluctuations in the ultraviolet background (UVB), where the level of UVB fluctuations are set by the mean free path of ionizing photons ($\lambda_{\text{mfp}}$). The auto-correlation function of Ly$\alpha$ forest flux characterizes the strength and scale of transmission fluctuations and, as we show, is thus sensitive to $\lambda_{\text{mfp}}$. Recent measurements at $z \sim 6$ suggest an unexpected rapid evolution of $\lambda_{\text{mfp}}$ at $z>5.0$ which would leave a signature in the evolution of the auto-correlation function. For this forecast, we model mock Ly$\alpha$ forest data with properties similar to the XQR-30 extended data set at $5.4 \leq z \leq 6.0$. Due to cosmic variance, we investigate 100 mock data sets at each $z$. Additionally we look at ideal cases, where mock data matches model values of the auto-correlation function. For such ideal data with $\lambda_{\text{mfp}}=9.0$ cMpc at $z=6.0$, we recover $\lambda_{\text{mfp}}=12^{+6}_{-3}$ cMpc. This precision is comparable to direct measurements of $\lambda_{\text{mfp}}$ from the stacking of quasar spectra beyond the Lyman limit. Hypothetical data with resolution over three times greater than XQR-30 data leads to a $\sim40\%$ reduction in the error bars over all $z$. The distribution of mock values of the auto-correlation function in this work is highly non-Gaussian for high-$z$, which should caution work with other statistics of the high-$z$ Ly$\alpha$ forest against making this assumption. We use a rigorous statistical method to pass an inference test, however future work on non-Gaussian methods will enable higher precision measurements. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure Figures/forward_model_flux_z54_mfp32_F4_skewer_0.pdf</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.09230-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.09230) | **Millimeter Gap Contrast as a Probe for Turbulence Level in  Protoplanetary Disks**  |
|| Yao Liu, et al. -- incl., <mark>Mario Flock</mark> |
|*Appeared on*| *2022-08-22*|
|*Comments*| *13 pages, 7 figures, accepted for publication in Science China Physics, Mechanics & Astronomy*|
|**Abstract**| Turbulent motions are believed to regulate angular momentum transport and influence dust evolution in protoplanetary disks. Measuring the strength of turbulence is challenging through gas line observations because of the requirement for high spatial and spectral resolution data, and an exquisite determination of the temperature. In this work, taking the well-known HD 163296 disk as an example, we investigated the contrast of gaps identified in high angular resolution continuum images as a probe for the level of turbulence. With self-consistent radiative transfer models, we simultaneously analyzed the radial brightness profiles along the disk major and minor axes, and the azimuthal brightness profiles of the B67 and B100 rings. By fitting all the gap contrasts measured from these profiles, we constrained the gas-to-dust scale height ratio $\Lambda$ to be $3.0_{-0.8}^{+0.3}$, $1.2_{-0.1}^{+0.1}$ and ${\ge}\,6.5$ for the D48, B67 and B100 regions, respectively. The varying gas-to-dust scale height ratios indicate that the degree of dust settling changes with radius. The inferred values for $\Lambda$ translate into a turbulence level of $\alpha_{\rm turb}\,{<}\,3\times10^{-3}$ in the D48 and B100 regions, which is consistent with previous upper limits set by gas line observations. However, turbulent motions in the B67 ring are strong with $\alpha_{\rm turb}\,{\sim}1.2\,{\times}\,10^{-2}$. Due to the degeneracy between $\Lambda$ and the depth of dust surface density drops, the turbulence strength in the D86 gap region is not constrained. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error [Errno 2] No such file or directory: 'gs'</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2208.09335.md
    + _build/html/tmp_2208.09335/./figures/residuals_tworanges.png
    + _build/html/tmp_2208.09335/./figures/c_vs_dens_M10_magRanges.png
    + _build/html/tmp_2208.09335/./figures/obs_and_model_four_panels.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\Gaia}{{\it Gaia}\xspace}$
$\newcommand{\cat}{\ensuremath{\mathcal{C}}}$
$\newcommand{\vq}{\ensuremath{\mathbf{q}}}$
$\newcommand{\cca}{Center for Computational Astrophysics, Flatiron Institute, 162 Fifth Ave, New York, NY 10010, USA}$
$\newcommand{\ccpp}{Center for Cosmology and Particle Physics, Department of Physics, New~York~University, 726~Broadway, New~York, NY 10003, USA}$
$\newcommand{\mpia}{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\newcommand{\leiden}{Leiden Observatory, Leiden University, Niels Bohrweg 2, 2333 CA Leiden, The Netherlands}$
$\newcommand{\cambridge}{Institute of Astronomy, University of Cambridge, Madingley Road, Cambridge CB3 0HA, United Kingdom}$
$\newcommand{\oxford}{Magdalen College, Oxford University, Oxford OX1 4AU, United Kingdom }$
$\newcommand{\torino}{INAF - Osservatorio Astrofisico di Torino, Strada Osservatorio 20, Pino Torinese 10025 Torino, Italy}$
$\newcommand{\monash}{School of Physics and Astronomy, Monash University, VIC 3800, Australia}$
$\newcommand{\harvard}{Department of Physics, Harvard University, 17 Oxford St., Cambridge, MA 02138, USA}$
$\newcommand{\CFA}{Harvard-Smithsonian Center for Astrophysics, 60 Garden St., Cambridge, MA 02138, USA}$
$\newcommand{\Monash}{School of Physics \& Astronomy, Monash University, Clayton 3800, Victoria, Australia}$
$\newcommand{\AstroTD}{Centre of Excellence for Astrophysics in Three Dimensions (ASTRO-3D), Melbourne, Victoria, Australia}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\Gaia$}{{\it Gaia}$\xspace$}$
$\newcommand{$\cat$}{$\ensuremath${\mathcal{C}}}$
$\newcommand{$\vq$}{$\ensuremath${\mathbf{q}}}$
$\newcommand{$\cca$}{Center for Computational Astrophysics, Flatiron Institute, 162 Fifth Ave, New York, NY 10010, USA}$
$\newcommand{$\ccpp$}{Center for Cosmology and Particle Physics, Department of Physics, New~York~University, 726~Broadway, New~York, NY 10003, USA}$
$\newcommand{$\mpia$}{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\newcommand{$\leiden$}{Leiden Observatory, Leiden University, Niels Bohrweg 2, 2333 CA Leiden, The Netherlands}$
$\newcommand{$\cambridge$}{Institute of Astronomy, University of Cambridge, Madingley Road, Cambridge CB3 0HA, United Kingdom}$
$\newcommand{$\oxford$}{Magdalen College, Oxford University, Oxford OX1 4AU, United Kingdom }$
$\newcommand{$\torino$}{INAF - Osservatorio Astrofisico di Torino, Strada Osservatorio 20, Pino Torinese 10025 Torino, Italy}$
$\newcommand{$\monash$}{School of Physics and Astronomy, Monash University, VIC 3800, Australia}$
$\newcommand{$\harvard$}{Department of Physics, Harvard University, 17 Oxford St., Cambridge, MA 02138, USA}$
$\newcommand{$\CFA$}{Harvard-Smithsonian Center for Astrophysics, 60 Garden St., Cambridge, MA 02138, USA}$
$\newcommand{$\Monash$}{School of Physics \& Astronomy, Monash University, Clayton 3800, Victoria, Australia}$
$\newcommand{$\AstroTD$}{Centre of Excellence for Astrophysics in Three Dimensions (ASTRO-3D), Melbourne, Victoria, Australia}$</div>



<div id="title">

#  DR3 selection function

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2208.09335-b31b1b.svg)](https://arxiv.org/abs/2208.09335)<mark>Appeared on: 2022-08-22</mark> - _to be submitted to A&A_

</div>
<div id="authors">

<mark><mark>Tristan Cantat-Gaudin</mark></mark>, et al. -- incl., <mark><mark>Morgan Fouesneau</mark></mark>, <mark><mark>Hans-Walter Rix</mark></mark>, <mark><mark>Anthony G. A. Brown</mark></mark>, <mark><mark>Gregory M. Green</mark></mark>

</div>
<div id="abstract">

**Abstract:** Interpreting and modelling astronomical catalogues requires an understanding of the catalogues' completeness or selection function: objects of what properties had a chance to end up in the catalogue. Here we set out to empirically quantify the completeness of the overall$\Gaia$DR3 catalogue. This task is not straightforward because$\Gaia$is the all-sky optical survey with the highest angular resolution to date and no consistent "ground truth" exists to allow direct comparisons.  However, well-characterised deeper imaging enables an empirical assessment of$\Gaia$'s$G$-band completeness across parts of the sky.  On this basis, we devised a simple analytical completeness model of$\Gaia$as a function of the observed$G$magnitude and position over the sky, which accounts for both the effects of crowding and the complex$\Gaia$scanning law. Our model only depends on a single quantity: the median magnitude$M_{10}$in a patch of the sky of catalogued sources with\texttt{astrometric\_matched\_transits}$\leq 10$.$M_{10}$reflects elementary completeness decisions in the$\Gaia$pipeline and is computable from the$\Gaia$DR3 catalogue itself and therefore applicable across the whole sky. We calibrate our model using the Dark Energy Camera Plane Survey (DECaPS) and test its predictions against Hubble Space Telescope observations of globular clusters. We find that our model predicts$\Gaia$'s completeness values to a few per cent ({\it RMS}) across the sky. We make the model available as a part of the\texttt{gaiasf}Python package built and maintained by the GaiaUnlimited project\thanks{\url{https://github.com/gaia-unlimited/gaiasf}}.

</div>

<div id="div_fig1">

<img src="tmp_2208.09335/./figures/residuals_tworanges.png" alt="Fig7" width="100%"/>

**Figure 7. -**  Mean completeness residuals, i.e. model predicted minus observed completeness,  as a function of magnitude, for patches in two different ranges of $M_{10}$. The shaded areas correspond to the 16th to 84th percentile and 5th to 95th percentile intervals. Expectedly, the residuals are highest near 50\% completeness and smaller in the highly complete or dramatically incomplete regime.
     (*fig:residuals_tworanges*)

</div>
<div id="div_fig2">

<img src="tmp_2208.09335/./figures/c_vs_dens_M10_magRanges.png" alt="Fig3" width="100%"/>

**Figure 3. -**  Completeness of $\Gaia$ relative to the DECaPS survey, which is taken as "ground truth", in four magnitude ranges, computed in 3000 distinct patches across the DECaPS footprint. This completeness is shown as a function of $\Gaia$ source density (left) or $M_{10}$(right):  $\Gaia$ source density is a poor predictor of completeness, while the $M_{10}$ parameter --  the median magnitude of catalogued sources with  \texttt{astrometric\_matched\_transits}$\leq 10$ in a surrounding patch of the sky --  is an excellent completeness predictor. $M_{10}$ combines the impact of source density and scanning law, as demonstrated e.g. in Figure \ref{fig:mapM10}.
     (*fig:c_vs_M10_magRanges*)

</div>
<div id="div_fig3">

<img src="tmp_2208.09335/./figures/obs_and_model_four_panels.png" alt="Fig5" width="100%"/>

**Figure 5. -** $\Gaia$'s catalogue completeness $S(G | M_{10})$ as a function of $G$ magnitude for a given completeness parameter $M_{10}$. Top left: empirically determined $\Gaia$$G$ completeness derived from the comparison with DECaPS photometry in thirteen DECaPS patches that are each colour-coded by their $M_{10}$. Top right: model completeness, from Eq. \ref{eq:sigmoid}, for the corresponding $M_{10}$ values. Bottom left: empirically-determined completeness derived from from the comparison with DECaPS photometry in 105 DECaPS patches with $M_{10}$=19.4, 19.8,20.2, 20.6, and 21 (within 0.01 mag). The dispersion at a given $M_{10}$ corresponds to the scatter seen in Fig. \ref{fig:c_vs_M10_magRanges}(right panel) and the residuals in Fig. \ref{fig:residuals_tworanges}. Bottom right: model completeness for these five values of $M_{10}$.
     (*fig:obs_and_model*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

20  publications in the last 7 days.
	 _build/html/2208.09335.md
	 _build/html/2208.08872.md
	 _build/html/2204.06393.md
	 _build/html/2204.03335.md
	 _build/html/2204.03253.md
	 _build/html/2204.02998.md
	 _build/html/2204.02109.md
	 _build/html/2204.02017.md
	 _build/html/2204.01824.md
	 _build/html/2204.01758.md
	 _build/html/2204.01245.md
	 _build/html/2204.00793.md
	 _build/html/2204.00342.md
	 _build/html/2203.16959.md
	 _build/html/2203.16856.md
	 _build/html/2203.16735.md
	 _build/html/2203.16734.md
	 _build/html/2203.16504.md
	 _build/html/2203.15822.md
	 _build/html/2203.15811.md


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers